### MAPBOX API CALL FOR ISOCHRONES & SAVING TO FILES

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import os, time, requests, json
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio
from rasterio.mask import mask
from shapely.geometry import mapping
from pathlib import Path
import folium
from IPython.display import display
from plotly.subplots import make_subplots

In [5]:
# SETTINGS  
MAPBOX_TOKEN   = "token"   
RASTER_FP      = "/Users/riainfitzsimons/Desktop/brooklyn_museum_geo/data/raw_data/LandScan/LandScanConusNight.tif"
PROFILE        = "driving"
MINUTES        = 30
OUT_DIR        = Path(".")                     # current directory
DELAY_SEC      = 0.35                          # courtesy pause; 300 req/min limit



# ORIGIN LIST  
origins = [
    {"name": "art_institute_of_chicago", "lat": 41.879603071247, "lon": -87.62265081095006},
    {"name": "baltimore_museum_of_art",  "lat": 39.326092077899354, "lon": -76.61954755154906},
    {"name": "museum_of_fine_arts_boston", "lat": 42.33932545826515, "lon": -71.09404800351642},
    {"name": "national_portrait_gallery", "lat": 38.89773482278652, "lon": -77.02308337669994},
    {"name": "museum_of_modern_art_nyc",  "lat": 40.761710723742574, "lon": -73.9778901686774},
]



def fetch_isochrone(lon, lat, profile="driving", minutes=30, token=""):
    url = (
        f"https://api.mapbox.com/isochrone/v1/mapbox/{profile}/"
        f"{lon},{lat}"
        f"?contours_minutes={minutes}&polygons=true&access_token={token}"
    )
    r = requests.get(url)
    r.raise_for_status()
    return r.json()

def clip_raster_sum(raster_fp, poly_gdf):
    with rasterio.open(raster_fp) as src:
        poly_proj = poly_gdf.to_crs(src.crs)
        geom = [mapping(poly_proj.iloc[0].geometry)]
        out_img, _ = mask(src, geom, crop=True, filled=False)
    band = out_img[0].astype("float32")
    band[out_img.mask[0]] = np.nan
    return np.nansum(band)   # total population in polygon

results = []

for origin in origins:
    name = origin["name"]
    lat, lon = origin["lat"], origin["lon"]
    print(f"⇢ Processing {name} …")

    # 1. Fetch isochrone 
    isojson = fetch_isochrone(lon, lat, PROFILE, MINUTES, MAPBOX_TOKEN)
    gdf_iso = gpd.GeoDataFrame.from_features(isojson["features"], crs="EPSG:4326")

    iso_fp = OUT_DIR / f"{name}_30min_iso.geojson"
    gdf_iso.to_file(iso_fp, driver="GeoJSON")

    # 2. Clip raster & sum population 
    pop_total = clip_raster_sum(RASTER_FP, gdf_iso)

    # 3. Save clipped raster for record
    with rasterio.open(RASTER_FP) as src:
        geom = [mapping(gdf_iso.to_crs(src.crs).iloc[0].geometry)]
        out_img, out_transform = mask(src, geom, crop=True, filled=True, nodata=src.nodata)
        out_meta = src.meta.copy()
        out_meta.update({"height": out_img.shape[1], "width": out_img.shape[2], "transform": out_transform})
    tif_fp = OUT_DIR / f"{name}_30min_clipped.tif"
    with rasterio.open(tif_fp, "w", **out_meta) as dest:
        dest.write(out_img)

    # 4. Append
    results.append({"origin": name, "total_pop": int(pop_total)})

    time.sleep(DELAY_SEC)   # its good to be polite

# Summary CSV 
df = pd.DataFrame(results)
csv_fp = OUT_DIR / "isochrone_population_summary.csv"
df.to_csv(csv_fp, index=False)

print("\n✔ Done!  Population totals:")
print(df.to_string(index=False))
print(f"\nCSV written to: {csv_fp.resolve()}")

⇢ Processing art_institute_of_chicago …
⇢ Processing baltimore_museum_of_art …
⇢ Processing museum_of_fine_arts_boston …
⇢ Processing national_portrait_gallery …
⇢ Processing museum_of_modern_art_nyc …

✔ Done!  Population totals:
                    origin  total_pop
  art_institute_of_chicago    1205991
   baltimore_museum_of_art    1225603
museum_of_fine_arts_boston    1626418
 national_portrait_gallery    2068526
  museum_of_modern_art_nyc    2603282

CSV written to: /Users/riainfitzsimons/Desktop/brooklyn_museum_geo/notebooks/isochrone_population_summary.csv


### VISUALIZING ISOCHRONES

In [6]:

for rec in results:
    name   = rec["origin"]
    iso_fp = OUT_DIR / f"{name}_30min_iso.geojson"
    html_fp = OUT_DIR / f"{name}_30min_map.html"

    gdf_iso = gpd.read_file(iso_fp)
    centroid = gdf_iso.geometry.iloc[0].centroid

    # Base map
    m = folium.Map(location=[centroid.y, centroid.x],
                   zoom_start=11,
                   tiles="cartodbpositron")

    # Isochrone polygon layer (yellow outline)
    folium.GeoJson(
        gdf_iso,
        name="30‑min driving isochrone",
        style_function=lambda _:{
            "color": "yellow",
            "weight": 3,
            "fill": False
        }
    ).add_to(m)

    # Optional: pop‑up at centre with population total
    folium.Marker(
        location=[centroid.y, centroid.x],
        icon=folium.Icon(color="blue", icon="info-sign"),
        popup=f"<b>Total night‑time population<br>within 30 min:</b><br>{rec['total_pop']:,}"
    ).add_to(m)

    folium.LayerControl().add_to(m)
    m.save(html_fp)
    print(f"🗺  Saved interactive map →  {html_fp}")

print("\nEmbed with e.g. :")
print('<iframe src="museum_of_modern_art_nyc_30min_map.html" width="100%" height="600" frameborder="0"></iframe>')


🗺  Saved interactive map →  art_institute_of_chicago_30min_map.html
🗺  Saved interactive map →  baltimore_museum_of_art_30min_map.html
🗺  Saved interactive map →  museum_of_fine_arts_boston_30min_map.html
🗺  Saved interactive map →  national_portrait_gallery_30min_map.html
🗺  Saved interactive map →  museum_of_modern_art_nyc_30min_map.html

Embed with e.g. :
<iframe src="museum_of_modern_art_nyc_30min_map.html" width="100%" height="600" frameborder="0"></iframe>


In [7]:


for rec in results:
    name   = rec["origin"]
    iso_fp = OUT_DIR / f"{name}_30min_iso.geojson"
    
    gdf_iso  = gpd.read_file(iso_fp)
    centroid = gdf_iso.geometry.iloc[0].centroid
    
    m = folium.Map(location=[centroid.y, centroid.x],
                   zoom_start=11,
                   tiles="cartodbpositron")
    
    folium.GeoJson(
        gdf_iso,
        name="30‑min driving isochrone",
        style_function=lambda _:{
            "color": "yellow",
            "weight": 3,
            "fill": False
        }
    ).add_to(m)
    
    folium.Marker(
        location=[centroid.y, centroid.x],
        icon=folium.Icon(color="blue", icon="info-sign"),
        popup=f"<b>Total night‑time population<br>within 30 min:</b><br>{rec['total_pop']:,}"
    ).add_to(m)
    
    folium.LayerControl().add_to(m)
    
    print(f"▼ {name.replace('_', ' ').title()}")
    display(m)


▼ Art Institute Of Chicago


▼ Baltimore Museum Of Art


▼ Museum Of Fine Arts Boston


▼ National Portrait Gallery


▼ Museum Of Modern Art Nyc


### NORMALIZE METRICS BY POPULATION, SAVE TO CSV

In [8]:
# 1.  Bring population totals into a DataFrame 
pop_df = pd.DataFrame(results)

# 2.  Raw museum metrics 
raw = pd.DataFrame([
    {"origin": "art_institute_of_chicago",
     "attendance": 1_230_912,
     "gift_revenue": 18_500_000,
     "objects_purchased": 450},

    {"origin": "baltimore_museum_of_art",
     "attendance": 206_357,
     "gift_revenue": 3_200_000,
     "objects_purchased": 120},

    {"origin": "museum_of_fine_arts_boston",
     "attendance": 849_107,
     "gift_revenue": 12_000_000,
     "objects_purchased": 380},

    {"origin": "national_portrait_gallery",
     "attendance": 1_147_388,
     "gift_revenue": 15_000_000,
     "objects_purchased": 340},

    {"origin": "museum_of_modern_art_nyc",
     "attendance": 2_687_021,
     "gift_revenue": 45_000_000,
     "objects_purchased": 900},
])

# 3.  Merge with population totals 
df = raw.merge(pop_df, on="origin", how="left")

# 4.  Normalise per 1 000 residents 
for col in ["attendance", "gift_revenue", "objects_purchased"]:
    df[f"{col}_per_1k"] = df[col] / df["total_pop"] * 1_000

# 5.  Rank each normalised metric 
for col in ["attendance_per_1k", "gift_revenue_per_1k", "objects_purchased_per_1k"]:
    df[f"rank_{col}"] = df[col].rank(ascending=False, method="min").astype(int)

# 6.  Composite score (70 % attendance + 30 % revenue) 
#     Using normalised per‑1 k values
df["composite_score"] = (
    0.7 * df["attendance_per_1k"].rank(pct=True) + 
    0.3 * df["gift_revenue_per_1k"].rank(pct=True)
)

df["rank_composite"] = df["composite_score"].rank(ascending=False, method="min").astype(int)

# 7.  Save detailed metrics & rankings 
norm_csv = "isochrone_normalised_metrics.csv"
rank_csv = "isochrone_weighted_rankings.csv"

# a) full table
df.to_csv(norm_csv, index=False,
          columns=["origin", "total_pop",
                   "attendance", "attendance_per_1k", "rank_attendance_per_1k",
                   "gift_revenue", "gift_revenue_per_1k", "rank_gift_revenue_per_1k",
                   "objects_purchased", "objects_purchased_per_1k", "rank_objects_purchased_per_1k"])

# b) composite rankings only
df[["origin", "composite_score", "rank_composite"]].sort_values("rank_composite").to_csv(rank_csv, index=False)

print("✔ Normalised metric table →", norm_csv)
print("✔ Composite rankings      →", rank_csv)
df[["origin", "composite_score", "rank_composite"]].sort_values("rank_composite")


✔ Normalised metric table → isochrone_normalised_metrics.csv
✔ Composite rankings      → isochrone_weighted_rankings.csv


,origin,composite_score,rank_composite
4,museum_of_modern_art_nyc,1.00,1
0,art_institute_of_chicago,0.80,2
3,national_portrait_gallery,0.54,3
2,museum_of_fine_arts_boston,0.46,4
1,baltimore_museum_of_art,0.20,5


### VISUALIZE THE METRICS

In [9]:
base = Path("/Users/riainfitzsimons/Desktop/brooklyn_museum_geo/outputs")

# Load the files
df_metrics  = pd.read_csv(base / "isochrone_normalised_metrics.csv")
df_rankings = pd.read_csv(base / "isochrone_weighted_rankings.csv")

# Inspect dtypes
print("📊  Normalised metrics dtypes:")
print(df_metrics.dtypes, "\n")

print("🏆  Weighted rankings dtypes:")
print(df_rankings.dtypes)

📊  Normalised metrics dtypes:
origin                            object
total_pop                          int64
attendance                         int64
attendance_per_1k                float64
rank_attendance_per_1k             int64
gift_revenue                       int64
gift_revenue_per_1k              float64
rank_gift_revenue_per_1k           int64
objects_purchased                  int64
objects_purchased_per_1k         float64
rank_objects_purchased_per_1k      int64
dtype: object 

🏆  Weighted rankings dtypes:
origin              object
composite_score    float64
rank_composite       int64
dtype: object


In [14]:
# Load the population summary CSV
csv_path = "/Users/riainfitzsimons/Desktop/brooklyn_museum_geo/outputs/isochrone_population_summary.csv"
d = pd.read_csv(csv_path)

# Display data types
print("📄 Data types:")
print(d.dtypes)

# Optional: preview the data
d.head()

📄 Data types:
origin       object
total_pop     int64
dtype: object


,origin,total_pop
0,art_institute_of_chicago,1205991
1,baltimore_museum_of_art,1225603
2,museum_of_fine_arts_boston,1626418
3,national_portrait_gallery,2068526
4,museum_of_modern_art_nyc,2603282


In [17]:


# Load data
csv_path = "/Users/riainfitzsimons/Desktop/brooklyn_museum_geo/outputs/isochrone_population_summary.csv"
df = pd.read_csv(csv_path)

# Sort and clean
df = df.sort_values("total_pop", ascending=False).reset_index(drop=True)
df['label'] = df['origin'].str.replace('_', ' ').str.title()

# Create graduated alpha levels (opacity)
num_bars = len(df)
alphas = np.linspace(1.0, 0.5, num_bars)  # from opaque to semi-transparent

# Baby blue RGBA
colors = [f"rgba(137, 207, 240, {alpha})" for alpha in alphas]  # baby blue RGBA

# Plot
fig = go.Figure()
fig.add_trace(go.Bar(
    x=df["label"],
    y=df["total_pop"],
    text=df["total_pop"],
    textposition='outside',
    marker_color=colors
))

# Styling
fig.update_layout(
    title="Night-Time Population Within 30-Min Isochrone",
    template="simple_white",
    title_font=dict(size=16),
    showlegend=False,
    xaxis_title=None,
    yaxis_title="Total Population",
    xaxis_tickangle=-15,
    margin=dict(t=60, b=40, l=40, r=40)
)

fig.show()


In [13]:
# 1. Load data 
base = Path("/Users/riainfitzsimons/Desktop/brooklyn_museum_geo/outputs")
df = (
    pd.read_csv(base / "isochrone_normalised_metrics.csv")
      .merge(pd.read_csv(base / "isochrone_weighted_rankings.csv"), on="origin")
      .sort_values("rank_composite")          # best → worst
      .reset_index(drop=True)
)

# Friendly names for y‑axis labels
df["label"] = df["origin"].str.replace("_", " ").str.title()

# 2. Build 2×2 subplot grid 
fig = make_subplots(
    rows=2, cols=2,
    shared_yaxes=True,
    horizontal_spacing=0.12,
    vertical_spacing=0.13,
    subplot_titles=(
        "Attendance / 1 000 residents",
        "Gift‑store revenue / 1 000 residents (USD)",
        "Objects purchased / 1 000 residents",
        "Weighted composite score"
    )
)

# Helper to add a bar trace
def add_bar(row, col, x, title, color):
    fig.add_trace(
        go.Bar(
            x=df[x],
            y=df["label"],
            orientation="h",
            marker_color=color,
            hovertemplate="%{y}<br>%{x:,.2f}<extra></extra>",
        ),
        row=row, col=col
    )

add_bar(1, 1, "attendance_per_1k",         "Attendance / 1 000",      "#2a9d8f")
add_bar(1, 2, "gift_revenue_per_1k",       "Revenue / 1 000",         "#e9c46a")
add_bar(2, 1, "objects_purchased_per_1k",  "Objects / 1 000",         "#f4a261")
add_bar(2, 2, "composite_score",           "Composite score",         "#264653")

# 3. Layout tweaks
fig.update_layout(
    template="plotly_white",
    height=620,
    showlegend=False,
    margin=dict(l=120, r=40, t=60, b=40),
    font=dict(family="Helvetica Neue, sans-serif", size=12),
)

# Title styling
fig.update_annotations(font_size=13, font_color="#333")

# Axes styling
for i in range(1, 5):
    fig.update_xaxes(showgrid=False, zeroline=False, row=(i-1)//2+1, col=(i-1)%2+1)
    fig.update_yaxes(autorange="reversed", showgrid=False, row=(i-1)//2+1, col=(i-1)%2+1)

fig.show()